## Prompt a LLM (e.g. gpt-3.5-turbo) to get a Definition and Guidelines for each NE type of the dataset at hand

In [2]:
from dotenv import load_dotenv
from openai import OpenAI
import tiktoken 

import json
import sys
import os

In [5]:
# Load API key from .env
load_dotenv('./.env')
print(os.environ.get('OPENAI_API_KEY') is not None)

True


In [6]:
client = OpenAI()

In [7]:
gpt_model = "gpt-3.5-turbo-1106"

In [11]:
dataset_name = "Multinerd_it"
with open(f"./sentences_x_NE/{dataset_name}.json", 'r') as file:
        sentences_per_ne_type = json.load(file)

ne_types_list = list(sentences_per_ne_type.keys())

print("NE types:")
print(len(ne_types_list))
print(ne_types_list)

NE types:
15
['ANIM', 'BIO', 'CEL', 'DIS', 'EVE', 'FOOD', 'INST', 'LOC', 'MEDIA', 'MYTH', 'ORG', 'PER', 'PLANT', 'TIME', 'VEHI']


In [13]:
with open(os.path.join(f'../../datasets/multinerd', 'extended_labels_map.jsonl')) as fp:
    extended_labels_map = json.load(fp)

In [17]:
sentences_per_ne_type_w_real_name = {}
for ne in ne_types_list:
    sentences = sentences_per_ne_type[ne]
    sentences_per_ne_type_w_real_name[ne] = {
        "real_name": extended_labels_map[ne],
        "sentences": sentences
    }

print(sentences_per_ne_type_w_real_name)
sentences_per_ne_type = sentences_per_ne_type_w_real_name

{'ANIM': {'real_name': 'animale', 'sentences': [{'sentence': 'Tra i mammiferi , è presente il rinolofo euriale ( " Rhinolophus euryale " ) , un chirottero .', 'target_words_in_it': ['Rhinolophus euryale'], 'occurrences_count': 1}, {'sentence': 'Il fiume in queste zone ospita una grande popolazioni di ippopotami .', 'target_words_in_it': ['ippopotami'], 'occurrences_count': 1}, {'sentence': 'Tra le specie maggiormente predate i primati di piccola taglia come il Lori lento .', 'target_words_in_it': ['Lori lento'], 'occurrences_count': 1}]}, 'BIO': {'real_name': 'entità biologica', 'sentences': [{'sentence': 'Le macchie nere sulle foglie sono causate dal Colletotrichum .', 'target_words_in_it': ['Colletotrichum'], 'occurrences_count': 1}, {'sentence': 'Un piccolo numero di lieviti come la " Candida albicans ", può causare infezioni nell\'uomo.', 'target_words_in_it': ['Candida albicans'], 'occurrences_count': 1}, {'sentence': 'Il genere è stato introdotto nel 1998 come " genere dei virus 

In [14]:
print(extended_labels_map)

{'PER': 'persona', 'ORG': 'organizzazione', 'LOC': 'località', 'ANIM': 'animale', 'BIO': 'entità biologica', 'CEL': 'corpo celeste', 'DIS': 'malattia', 'EVE': 'evento', 'FOOD': 'cibo', 'INST': 'strumento', 'MEDIA': 'media', 'MYTH': 'entità mitologica', 'PLANT': 'pianta', 'TIME': 'tempo', 'VEHI': 'veicolo'}


In [10]:
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-1106"):
    """Return the number of tokens used in the input prompt."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model in {
        "gpt-3.5-turbo-0613",
        "gpt-3.5-turbo-16k-0613",
        "gpt-4-0314",
        "gpt-4-32k-0314",
        "gpt-4-0613",
        "gpt-4-32k-0613",
        }:
        tokens_per_message = 3
        tokens_per_name = 1
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = 4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
        tokens_per_name = -1  # if there's a name, the role is omitted
    elif "gpt-3.5-turbo" in model:
        # print("Warning: gpt-3.5-turbo may update over time. Returning num tokens assuming gpt-3.5-turbo-0613.")
        return num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613")
    elif "gpt-4" in model:
        print("Warning: gpt-4 may update over time. Returning num tokens assuming gpt-4-0613.")
        return num_tokens_from_messages(messages, model="gpt-4-0613")
    else:
        raise NotImplementedError(
            f"""num_tokens_from_messages() is not implemented for model {model}. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens."""
        )
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens

# PROMPT formulation

System message

In [ ]:
system_message = "You are a helpful NER data annotator designed to output JSON."  # OpenAI documentations suggest to write also here to output JSON

creating 1st 'user_prompt' using 'location' as example 

In [ ]:
ne_type = 'location'
#related_data = sentences_per_ne_type[ne_type]

#example_prompt = data_handler_cross_NER.generate_structured_prompt(related_data)

example_prompt = "Named Entity: 'location'. Examples: [{'sentence': 'He was awarded honorary degree from the University of Cambridge in Cambridge , UK , and the Copley Medal of the Royal Society in 1906 .', 'entities': ['Cambridge']}, {'sentence': 'According to onlookers, the communicant told the priest: \"You're in Ireland now\".', 'entities': ['Ireland']}, {'sentence': 'The Big Bend Country is part of the larger Columbia Country , which includes the Columbia Valley and upper Arrow Lakes .', 'entities': ['Arrow Lakes', 'Columbia Country', 'Big Bend Country', 'Columbia Valley']}]. Hints: You should not label 'here', 'there' and similar as 'location' entities.\nInstructions: 1. Provide a concise definition for the named entity 'location' in the context of NER. 2. Provide guidelines by specifying what entities should not be labeled as 'location' and include potential pitfalls to avoid. Go beyond generic terms and delve into nuanced scenarios. Be explicit about potential ambiguities and provide guidance on distinguishing 'location' from similar entities.\nOutput in JSON format: {\"Definition\": \"\", \"Guidelines\": \"\"}."

print("example_prompt:\n")
print(example_prompt)

What would look like a desired gpt's answer

In [ ]:
example_answer = "{\"Definition\": \"\'location\' denotes geographic entities, such as cities, countries, and landmarks, that represent specific places on Earth.\", \"Guidelines\": \"Do not label abstract references. Be cautious with ambiguous terms like 'Paris Hilton' (person with a location name) or 'Amazon' (both a company and a river)." # Be cautious with terms that may refer to both people and places, like 'Jordan' (person or country).\"}"
print("example answer:")
print(example_answer)

The real prompt for the desired NE 

In [ ]:
ne_type = 'Advisors.GENERIC_CONSULTING_COMPANY'  # pick any NE
ex_sentences = sentences_per_ne_type[ne_type]['sentences']
ex_sentences_json = []
for exsent in ex_sentences:
    ex_sentences_json.append({'sentence': exsent['sentence'], 'entities': exsent['target_words_in_it']})
real_prompt = f"Now do the same for the Named Entity: \'{sentences_per_ne_type[ne_type]['real_name']}\', Examples: {ex_sentences_json}"

print("real prompt: ")
print(real_prompt)

In [ ]:
messages_to_send = [
    {
        "role": "system",
        "content": system_message
    },
    {
        "role": "user",
        "content": example_prompt
    },
    {
        "role": "system",
        "content": example_answer
    },
    {
        "role": "user",
        "content": real_prompt
    }
]

In [ ]:
n_input_tokens = num_tokens_from_messages(messages_to_send, gpt_model)
print(f"The input prompt has length in tokens: {n_input_tokens}")

In [ ]:
print(messages_to_send)

### Pack it in 1 function

In [32]:
def get_json_definition_for_ne(named_entity, sentences_per_ne_type, gpt_model_name, max_output_tokens):
    # system message
    system_message = "Sei un esperto annotatore per la Named Entity Recognition, progettato per produrre output in formato JSON."
    # user prompt example on location
    example_prompt = "Tipo di entità: 'luogo'. Esempi: [{'frase': 'George Washington, nato in Virginia, è stato il primo presidente degli USA.', 'entità': ['Virginia']}, {'frase': 'Sono stato molte volte in vacanza a Napoli, in Italia', 'entità': ['Napoli', 'Italia']}, {'frase': 'Siamo andati in vacanza a Ginevra e abbiamo visitato l'ONU.', 'entità': ['Ginevra']}]. \nIstruzioni: 1. Fornisci una breve definizione per il tipo di entità 'luogo' nel contesto della NER. 2. Fornisci linee guida specificando quali entità non dovrebbero essere etichettate come 'luogo' e includi esempi di potenziali insidie. Vai oltre i termini generici e approfondisci scenari dettagliati. Sii esplicito riguardo alle potenziali ambiguità e fornisci indicazioni su come distinguere 'luogo' da entità simili.\nOutput in formato JSON: {\"Definition\": \"\", \"Guidelines\": \"\"}."
    # example answer
    example_answer = "{\"Definition\": \"\'luogo\' denota luoghi geografici come città, paesi, stati, regioni, continenti, punti di interesse naturale, e indirizzi specifici.\", \"Guidelines\": \"Assicurati di non confondere i nomi di luoghi con altre entità. Ad esempio, 'Washington', potrebbe riferirsi alla città di Washington D.C. o al presidente George Washington, quindi considera attentamente il contesto. Escludi nomi di periodi storici, eventi o concetti astratti che non rappresentano luoghi fisici. Ad esempio, 'nel Rinascimento' è un periodo storico, non un luogo geografico.\"}"
    # real prompt
    exemplary_data = sentences_per_ne_type[named_entity]
    ex_sentences = exemplary_data['sentences']
    real_ne_name = exemplary_data['real_name']
    ex_sentences_json = []
    for exsent in ex_sentences:
        ex_sentences_json.append({'frase': exsent['sentence'], 'entità': exsent['target_words_in_it']})
    real_prompt = f"Ora fai lo stesso per il tipo di entità: \'{real_ne_name}\', Esempi: {ex_sentences_json}"
    if 'Hints' in exemplary_data and exemplary_data['Hints'] != "":
        real_prompt += f". Suggerimenti: {exemplary_data['Hints']}\n"

    # message to send
    messages_to_send = [
    {
        "role": "system",
        "content": system_message
    },
    {
        "role": "user",
        "content": example_prompt
    },
    {
        "role": "system",
        "content": example_answer
    },
    {
        "role": "user",
        "content": real_prompt
    }
    ]

    n_input_tokens = num_tokens_from_messages(messages_to_send, gpt_model)
    print(messages_to_send)

    #for message in messages_to_send:
    #print(message)
    
    if n_input_tokens > 1000:
        raise ValueError(f"Too many input tokens in messages_to_send: {len(messages_to_send)}, {messages_to_send}")

    print(f"Sending prompt for NE: {named_entity}...")
    
    completion = client.chat.completions.create(
        messages=messages_to_send,
        model=gpt_model_name,
        max_tokens=max_output_tokens,
        response_format={ "type": "json_object" }
    )

    choice = completion.choices[0]
    finish_reason = choice.finish_reason
    message = choice.message.content

    #if finish_reason == 'length' and message[-2:] != "\"}":
        #message += "\"}"

    return {"ne_tag": named_entity,
            "real_name": real_ne_name,
            "sentences_as_example": ex_sentences_json,
            "prompt_length": completion.usage.prompt_tokens,
            "output_length": completion.usage.completion_tokens,
            "gpt_DeG": message,
            "finish_reason": finish_reason
            }
    

In [29]:
ne_definition = get_json_definition_for_ne('ANIM', sentences_per_ne_type, gpt_model, max_output_tokens=300)
print(ne_definition)

[{'role': 'system', 'content': 'Sei un esperto annotatore per la Named Entity Recognition, progettato per produrre output in formato JSON.'}, {'role': 'user', 'content': 'Tipo di entità: \'luogo\'. Esempi: [{\'frase\': \'George Washington, nato in Virginia, è stato il primo presidente degli USA.\', \'entità\': [\'Virginia\']}, {\'frase\': \'Sono stato molte volte in vacanza a Napoli, in Italia\', \'entità\': [\'Napoli\', \'Italia\']}, {\'frase\': \'Siamo andati in vacanza a Ginevra e abbiamo visitato l\'ONU.\', \'entità\': [\'Ginevra\']}]. \nIstruzioni: 1. Fornisci una breve definizione per il tipo di entità \'luogo\' nel contesto della NER. 2. Fornisci linee guida specificando quali entità non dovrebbero essere etichettate come \'luogo\' e includi esempi di potenziali insidie. Vai oltre i termini generici e approfondisci scenari dettagliati. Sii esplicito riguardo alle potenziali ambiguità e fornisci indicazioni su come distinguere \'luogo\' da entità simili.\nOutput in formato JSON: 

### get all definitions

In [33]:
definitions = []
for ne, exemplary_data in sentences_per_ne_type.items():
    try:
        ne_definition = get_json_definition_for_ne(ne, sentences_per_ne_type, gpt_model, max_output_tokens=250)
        definitions.append(ne_definition)
    except:
        with open(f"./{dataset_name}_err.jsonl", 'w') as f:
            json.dump(definitions, f, indent=2)
        print(f"Something went wrong while processing NE: {ne}")

with open(f"./{dataset_name}.jsonl", 'w') as f:
    json.dump(definitions, f, indent=2)

[{'role': 'system', 'content': 'Sei un esperto annotatore per la Named Entity Recognition, progettato per produrre output in formato JSON.'}, {'role': 'user', 'content': 'Tipo di entità: \'luogo\'. Esempi: [{\'frase\': \'George Washington, nato in Virginia, è stato il primo presidente degli USA.\', \'entità\': [\'Virginia\']}, {\'frase\': \'Sono stato molte volte in vacanza a Napoli, in Italia\', \'entità\': [\'Napoli\', \'Italia\']}, {\'frase\': \'Siamo andati in vacanza a Ginevra e abbiamo visitato l\'ONU.\', \'entità\': [\'Ginevra\']}]. \nIstruzioni: 1. Fornisci una breve definizione per il tipo di entità \'luogo\' nel contesto della NER. 2. Fornisci linee guida specificando quali entità non dovrebbero essere etichettate come \'luogo\' e includi esempi di potenziali insidie. Vai oltre i termini generici e approfondisci scenari dettagliati. Sii esplicito riguardo alle potenziali ambiguità e fornisci indicazioni su come distinguere \'luogo\' da entità simili.\nOutput in formato JSON: 

### END